To train this agent, click _Runtime_ and press _Run all_. Make sure you've enabled a free Tesla T4 GPU!

<div class="align-center">
<a href="https://github.com/openpipe/art"><img src="https://github.com/openpipe/art/raw/main/assets/ART_pill.png" height="50"></a>
<a href="https://discord.gg/zbBHRUpwf4"><img src="https://github.com/openpipe/art/raw/main/assets/Discord_pill.png" height="50"></a>
<a href="https://openpipe.ai/blog/art-e-mail-agent"><img src="https://github.com/openpipe/art/raw/main/assets/ART_E_pill.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [Github](https://github.com/openpipe/art).

</div>

<a href="https://art.openpipe.ai/"><img src="https://github.com/openpipe/art/raw/main/assets/Header_separator.png" height="5"></a>

This notebook shows how to train a Qwen 2.5 7B model to play Temporal Clue, a simplified version of the game Clue that relies on temporal reasoning. It will demonstrate how to set up a single-turn agent, how to train it, and how to evaluate it.

Completions will be logged to OpenPipe, and metrics will be logged to Weights & Biases.

You will learn how to construct an [agentic environment](#Environment), how to define a [rollout](#Rollout), and how to run a [training loop](#Loop).

SIDE NOTE: If you're curious about how Temporal Clue works and want to learn how we trained a model to beat o3-mini, check out the [blog post](https://openpipe.ai/blog/using-grpo-to-beat-o1-o3-mini-and-r1-on-temporal-clue)!


In [ ]:
!pip install "numpy<2.0.0"

### WARNING:

If you are running in Google Colab and installing numpy does not say "Requirement already satisfied: numpy<2.0.0" then click "Runtime" and "Restart Session."

In [ ]:
# make sure we're using numpy 1.*.*
import numpy as np

if (np.__version__).startswith("1."):
    print("Numpy version is 1.*.*, you're good to go!")
else:
    raise ValueError("Please restart your runtime using the above instructions!")

### Environment Variables

Later on in the notebook, we'll be creating a model that can automatically logs metrics to Weights & Biases. In order to do so, you'll need to provide your Weights & Biases API key as an environment variable.

You can also optionally initiate an OpenPipe client to report completions to a [dashboard](https://app.openpipe.ai) to get a feel for what the completions your model is generating look like, and how they change over time. Logging to OpenPipe is free, but is not required for training!


In [ ]:
import os

# Optional
WANDB_API_KEY = ""
if WANDB_API_KEY:
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY

# Optional
OPENPIPE_API_KEY = ""
if OPENPIPE_API_KEY:
    os.environ["OPENPIPE_API_KEY"] = OPENPIPE_API_KEY

### Installation

In [ ]:
%%capture
!uv pip install openpipe-art openpipe --prerelease allow --no-cache-dir

### Agentic Environment

<a name="Environment"></a>

ART allows your agent to learn by interacting with its environment. In this example, we'll create an environment in which the agent can play Temporal Clue.

Feel free to read as much or as little of this section's code as you'd like. The important thing to understand is that we're defining the rules of this agent's environment. In many cases, this will already be defined by the task you're trying to solve, but if you need to define a custom environment, this is how you do it.

In [ ]:
import art
import asyncio
from dotenv import load_dotenv
import json
import openai
import requests
import random
import re
from typing import TypedDict

load_dotenv()


class TemporalCluePuzzle(TypedDict):
    num_clues: int
    prompt: str
    solution: dict[str, str]


# download the puzzles from the github repo
puzzles_url = "https://raw.githubusercontent.com/openpipe/art/main/examples/data/temporal-clue/puzzles.json"
puzzles_response = requests.get(puzzles_url)

puzzles: list[TemporalCluePuzzle] = json.loads(puzzles_response.text)
val_puzzles = puzzles[:64]
test_puzzles = puzzles[64:128]
train_puzzles = puzzles[128:]
random.seed(42)
random.shuffle(train_puzzles)

### Creating a Model

Now that we've defined the rules of our environment, we can create a model that will learn to play Temporal Clue. We'll use a Qwen 2.5 7B model for this example. The `name` parameter will be associated with a wandb run, and the `base_model` parameter is the model that we'll be training a LoRA on top of.


In [ ]:
from art.local import LocalBackend

model = art.TrainableModel(
    name="001",
    project="temporal-clue",
    base_model="Qwen/Qwen2.5-7B-Instruct",
    _internal_config={"init_args": {"gpu_memory_utilization": 0.775}},
)
await model.register(LocalBackend(path="./.art"))

### Defining a Rollout

<a name="Rollout"></a>

A rollout is a single episode of an agent performing its task. It generates one or more trajectories, which are lists of messages and choices.

In this example, the rollout function loads a Temporal Clue prompt, and the agent gives its best guess at the solution. It then returns a trajectory which contains the `system` and `user` messages presented to the agent, as well as the `choices` that the agent made.

When the game is finished the `reward` for the agent's performance is calculated based on whether the agent's final answer matches the solution.

This rollout function will be called many times in parallel during each step of the training loop.


In [ ]:
async def rollout(
    model: art.Model, puzzle: TemporalCluePuzzle
) -> art.Trajectory:
    messages: art.Messages = [{"role": "user", "content": puzzle["prompt"]}]
    client = model.openai_client()
    chat_completion = await client.chat.completions.create(
        messages=messages, model=model.name
    )
    choice = chat_completion.choices[0]
    content = choice.message.content
    assert isinstance(content, str)
    num_correct = 0
    for key, value in puzzle["solution"].items():
        if matches := re.findall(rf"{key}\. ([A-Za-z \.:-]+)", content):
            match = matches[-1]
            if match.strip().lower() == value.lower():
                num_correct += 1
    reward = acc = num_correct / len(puzzle["solution"])
    return art.Trajectory(
        messages_and_choices=[*messages, choice], reward=reward, metrics={"acc": acc}
    )

<a name="Loop"></a>

### Training Loop

The training loop is where the magic happens. For each of the 1000 steps defined below, the rollout function will be called 2 times in parallel on each of the 64 validation puzzles, and 50 times in parallel on groups of 4 train puzzles. This means that 128 validation and 200 train games will be played at once. Each game will produce a trajectory, which will be used to update the model or record metrics.

The `gather_trajectory_groups` functions will wait for all of the trajectories to be generated, then we will delete all but the most recent checkpoint and train the model on the generated `train` trajectories.

Inference will be blocked until the training is complete.

In [ ]:
stride = 4
for i in range(await model.get_step(), 1_000):
    val_groups, train_groups = await asyncio.gather(
        art.gather_trajectory_groups(
            (
                art.TrajectoryGroup(rollout(model, puzzle) for _ in range(2))
                for puzzle in val_puzzles
            ),
            pbar_desc="val",
        ),
        art.gather_trajectory_groups(
            (
                art.TrajectoryGroup(rollout(model, puzzle) for _ in range(50))
                for puzzle in train_puzzles[i * stride : (i + 1) * stride]
            ),
            pbar_desc="train",
        ),
    )
    await model.log(val_groups)
    await model.delete_checkpoints()
    await model.train(
        train_groups,
        config=art.TrainConfig(learning_rate=5e-5),
    )

<div class="align-center">
<a href="https://github.com/openpipe/art"><img src="https://github.com/openpipe/art/raw/notebooks/assets/ART_pill.png" height="50"></a>
<a href="https://discord.gg/zbBHRUpwf4"><img src="https://github.com/openpipe/art/raw/notebooks/assets/Discord_pill.png" height="50"></a>
<a href="https://openpipe.ai/blog/art-e-mail-agent"><img src="https://github.com/openpipe/art/raw/main/assets/ART_E_pill.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [Github](https://github.com/openpipe/art).

</div>
